In [ ]:
import sys, os
import time

from makeDiambraEnv import *
from utils.policies import gamepadPolicy

timeDepSeed = int((time.time()-int(time.time()-0.5))*1000)

In [ ]:
base_path = "/home/yourUsername/DIAMBRAenvironment/"

diambraEnvKwargs = {}
diambraEnvKwargs["gameId"]          = "doapp"
diambraEnvKwargs["diambraEnv_path"] = base_path + "diambraEnvLib/"
diambraEnvKwargs["roms_path"]       = base_path + "roms/" # Absolute path to roms
diambraEnvKwargs["mame_path"]       = base_path + "mame/" # Absolute path to MAME executable

diambraEnvKwargs["mame_diambra_step_ratio"] = 6
diambraEnvKwargs["render"]      = True
diambraEnvKwargs["lock_fps"]    = True # Locks to 60 FPS
diambraEnvKwargs["sound"]       = diambraEnvKwargs["lock_fps"] and diambraEnvKwargs["render"]

# 1P
diambraEnvKwargs["player"] = "Random"
# 2P
#diambraEnvKwargs["player"] = "P1P2"

# Game specific
diambraEnvKwargs["difficulty"] = 3
diambraEnvKwargs["characters"]  = [["Random", "Random"], ["Random", "Random"]]
diambraEnvKwargs["charOutfits"] = [2, 2]

In [ ]:
# DIAMBRA gym kwargs
diambraGymKwargs = {}
diambraGymKwargs["P2brain"]               = None
diambraGymKwargs["continue_game"]         = 0.0
diambraGymKwargs["show_final"]            = False
diambraGymKwargs["gamePads"]              = [None, diambraGymKwargs["P2brain"]]
diambraGymKwargs["actionSpace"]           = ["multiDiscrete", "multiDiscrete"]
diambraGymKwargs["attackButCombinations"] = [True, True]
diambraGymKwargs["actBufLen"]             = 12

In [ ]:
# Wrappers kwargs
wrapperKwargs = {}
wrapperKwargs["hwc_obs_resize"]    = [256, 256, 1]
wrapperKwargs["normalize_rewards"] = True
wrapperKwargs["clip_rewards"]      = False
wrapperKwargs["frame_stack"]       = 6
wrapperKwargs["dilation"]          = 1
wrapperKwargs["scale"]             = True
wrapperKwargs["scale_mod"]         = 0

In [ ]:
# Additional Observations

#keyToAdd = None

keyToAdd = []
keyToAdd.append("ownActionsBuf")
if diambraEnvKwargs["player"] == "P1P2":
    keyToAdd.append("oppActionsBuf") # Only 2P

if diambraEnvKwargs["gameId"] != "tektagt": # DOA, SFIII, UMK3
    keyToAdd.append("ownHealth")
    keyToAdd.append("oppHealth")
else: # TEKTAG
    keyToAdd.append("ownHealth_1")
    keyToAdd.append("ownHealth_2")
    keyToAdd.append("oppHealth_1")
    keyToAdd.append("oppHealth_2")

keyToAdd.append("ownPosition")
keyToAdd.append("oppPosition")
#keyToAdd.append("ownWins")
#keyToAdd.append("oppWins")
keyToAdd.append("stage")
keyToAdd.append("characters")

In [ ]:
# Recording kwargs
trajRecKwargs = {}                                                          
trajRecKwargs["user_name"] = "Alex" #Your username
trajRecKwargs["file_path"] = os.path.join(base_path + "trajRecordings", diambraEnvKwargs["gameId"])
trajRecKwargs["ignore_p2"] = 0                                    
trajRecKwargs["commitHash"] = "0000000"

In [ ]:
envId = "Test"
env = make_diambra_env(diambraGym, env_prefix=envId, seed=timeDepSeed, 
                       diambra_kwargs=diambraEnvKwargs, 
                       diambra_gym_kwargs=diambraGymKwargs,
                       wrapper_kwargs=wrapperKwargs, 
                       key_to_add=keyToAdd,
                       traj_rec_kwargs=trajRecKwargs)

In [ ]:
# GamePad policy initialization
gamePad_policy = gamepadPolicy()
gamePad_policy.initialize(env.actionList)

In [ ]:
observation = env.reset()

In [ ]:
cumulativeEpRew = 0.0
cumulativeEpRewAll = []

maxNumEp = 4
currNumEp = 0

while currNumEp < maxNumEp:

    # 1P
    # Sampling actions from the action space
    #action = env.action_spaces[0].sample()
    # Using gamepad
    action = gamePad_policy.getActions()
    
    # 2P
    action2 = env.action_spaces[1].sample()
    if diambraEnvKwargs["player"] == "P1P2":
        action = np.append(action, action2)
        
    observation, reward, done, info = env.step(action)
        
    cumulativeEpRew += reward
    
    if np.any(done):
        currNumEp += 1
        print("Ep. # = ", currNumEp)
        print("Ep. Cumulative Rew # = ", cumulativeEpRew)
        cumulativeEpRewAll.append(cumulativeEpRew)
        cumulativeEpRew = 0.0

        observation = env.reset()

print("Mean cumulative reward = ", np.mean(cumulativeEpRewAll))    
print("Std cumulative reward = ", np.std(cumulativeEpRewAll))       
    
env.close()